In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import os

#Path till dataset
train_dir = "Data/CNN/train"
val_dir = "Data/CNN/val"
test_dir = "Data/CNN/test"

#Bild manipulering
datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=20, 
    zoom_range=0.2, 
    horizontal_flip=True,
    featurewise_center=True,  #Centrerar
    featurewise_std_normalization=True  #Standardiserar dataset
)

train_data = datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=32, class_mode="categorical")
val_data = datagen.flow_from_directory(val_dir, target_size=(224, 224), batch_size=32, class_mode="categorical")
test_data = datagen.flow_from_directory(test_dir, target_size=(224, 224), batch_size=32, class_mode="categorical")

#Laddar in för tränad EfficientNetB0
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  #Freeze base layers

#Lägger till nya klassifikations lager
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2, activation="relu")(x)
x = Dropout(0.3)(x) #First run 0.5
outputs = Dense(2, activation="softmax")(x)  #Klasser: Snowboarder & Skier

#Skapar modellen
model = Model(inputs=base_model.input, outputs=x)

#Kompilerar modellen
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5, epsilon=1e-7, clipnorm=1.0)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

#Tränar modellen
model.fit(train_data, validation_data=val_data, epochs=15)

#Finslipar (Unfreeze base model)
base_model.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5, epsilon=1e-7),
    loss="categorical_crossentropy",
    metrics=["accuracy"])
model.fit(train_data, validation_data=val_data, epochs=10)

#Evaluerar modellen
test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

#Sparar tränad modell
model.save("cnn_classifier.keras") #.h5 legacy format


Found 46 images belonging to 2 classes.
Found 23 images belonging to 2 classes.
Found 23 images belonging to 2 classes.


c:\Users\Felix Lemström\Desktop\School Stuffzerz\Datorseende\Projekt\Skidovervakaren\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
c:\Users\Felix Lemström\Desktop\School Stuffzerz\Datorseende\Projekt\Skidovervakaren\venv\Lib\site-packages\keras\src\legacy\preprocessing\image.py:1263: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(
c:\Users\Felix Lemström\Desktop\School Stuffzerz\Datorseende\Projekt\Skidovervakaren\venv\Lib\site-packages\keras\src\legacy\preprocessing\image.py:1273: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization

Epoch 1/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 12s 4s/step - accuracy: 0.3313 - loss: nan - val_accuracy: 0.1304 - val_loss: nan
Epoch 2/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.6387 - loss: nan - val_accuracy: 0.1304 - val_loss: nan
Epoch 3/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.5435 - loss: nan - val_accuracy: 0.1304 - val_loss: nan
Epoch 4/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 977ms/step - accuracy: 0.5435 - loss: nan - val_accuracy: 0.1304 - val_loss: nan
Epoch 5/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 972ms/step - accuracy: 0.5643 - loss: nan - val_accuracy: 0.1304 - val_loss: nan
Epoch 6/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 962ms/step - accuracy: 0.5851 - loss: nan - val_accuracy: 0.1304 - val_loss: nan
Epoch 7/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.5539 - loss: nan - val_accuracy: 0.1304 - val_loss: nan
Epoch 8/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.6149 - loss: nan - val_accuracy: 0.1304 - val_loss: nan
Epoch 9/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accur

: 